In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import jieba
import langconv

In [13]:
fname = 'D:\\python\\datasource\\Lession_10\\movie_comments.csv'

In [14]:
movie = pd.read_csv(fname, encoding = 'utf8')
len(movie)

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


261497

In [15]:
movie.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [16]:
for t in movie:
    print(t)

id
link
name
comment
star


In [56]:
comment_id = []
comment_comment = []
comment_star = []
for i, content in enumerate(movie['comment']):    
    comment, star = movie['comment'][i], movie['star'][i]
    star = str(star)
    if not star.isdigit(): continue
    text = str(comment)
    text = text.strip()
    if not text: continue
    text = langconv.Converter('zh-hans').convert(text)
    text = ''.join(re.findall(r'[\d|\w]+',text))
    text = ' '.join(jieba.cut(text))
    comment_id.append(id)    
    comment_comment.append(text)
    comment_star.append(star)
    if i % 2000 ==0:
        print('Saved ' + str(i) + ' lines')    

Saved 0 lines
Saved 2000 lines
Saved 4000 lines
Saved 6000 lines
Saved 8000 lines
Saved 10000 lines
Saved 12000 lines
Saved 14000 lines
Saved 16000 lines
Saved 18000 lines
Saved 20000 lines
Saved 22000 lines
Saved 24000 lines
Saved 26000 lines
Saved 28000 lines
Saved 30000 lines
Saved 32000 lines
Saved 34000 lines
Saved 36000 lines
Saved 38000 lines
Saved 40000 lines
Saved 42000 lines
Saved 44000 lines
Saved 46000 lines
Saved 48000 lines
Saved 50000 lines
Saved 52000 lines
Saved 54000 lines
Saved 56000 lines
Saved 58000 lines
Saved 60000 lines
Saved 62000 lines
Saved 64000 lines
Saved 66000 lines
Saved 68000 lines
Saved 70000 lines
Saved 72000 lines
Saved 74000 lines
Saved 76000 lines
Saved 78000 lines
Saved 80000 lines
Saved 82000 lines
Saved 84000 lines
Saved 86000 lines
Saved 88000 lines
Saved 90000 lines
Saved 92000 lines
Saved 94000 lines
Saved 96000 lines
Saved 98000 lines
Saved 100000 lines
Saved 102000 lines
Saved 104000 lines
Saved 106000 lines
Saved 108000 lines
Saved 110000 

In [57]:
cutFile = 'D:\python\datasource\Lession_10\cutResult.csv'

In [58]:
data = pd.DataFrame({'id':comment_id, 'comment':comment_comment, 'star':comment_star})
data.to_csv(cutFile,encoding = 'gb18030')

In [59]:
cut_comment = pd.read_csv(cutFile, encoding = 'gb18030')
cut_comment = cut_comment.loc[:, ~cut_comment.columns.str.contains('^Unnamed')]
for c in cut_comment:
    print(c)

comment
id
star


In [60]:
cut_comment = cut_comment.fillna(' ')

In [61]:
len(cut_comment)

261496

In [62]:
cut_comment.head()

,comment,id,star
0,吴京 意淫 到 了 脑残 的 地步 看 了 恶心 想 吐,260149,1
1,首映礼 看 的 太 恐怖 了 这个 电影 不讲道理 的 完全 就是 吴京 在 实现 他 这个...,260149,2
2,吴京 的 炒作 水平 不输 冯小刚 但小刚 至少 不会 用 主旋律 来 炒作 吴京 让 人 ...,260149,2
3,凭良心说 好 看到 不像 战狼 1 的 续集 完虐 湄公河 行动,260149,4
4,中二得 很,260149,1


In [81]:
comment_class = []
for i in cut_comment['star']:
    i = str(i)
    star = int(i)
    if star > 3:
        comment_class.append([1, 0])
    else:
        comment_class.append([0, 1])

In [65]:
len(comment_class)

261496

#### 训练集/测试集/验证集

In [69]:
index = np.random.choice(range(len(comment_class)),size = 12000, replace = False).tolist()
train_index = index[0:5000]
test_index = index[5000:10000]
valid_index = index[10000:]

In [82]:
train_text = [cut_comment['comment'][i] for i in train_index]
train_y = [comment_class[i] for i in train_index]

In [83]:
test_text = [cut_comment['comment'][i] for i in test_index]
test_y = [comment_class[i] for i in test_index]

In [84]:
valid_text = [cut_comment['comment'][i] for i in valid_index]
valid_y = [comment_class[i] for i in valid_index]

#### 文本向量化

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
stop_words = {}

In [79]:
feature_size = 10000
text = train_text
tfidf = TfidfVectorizer(max_features=feature_size, stop_words=stop_words)
fit_tfidf = tfidf.fit_transform(text)
train_x = fit_tfidf.toarray()
test_x = tfidf.transform(test_text).toarray()
valid_x = tfidf.transform(valid_text).toarray()

In [78]:
train_x.shape

(5000, 10000)

#### 神经网络分类

In [75]:
import tensorflow as tf

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [85]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [89]:
import time

In [96]:
batch_size = 128
hidden_nodes = 1024
num_labels = 2
#Graph define
g = tf.Graph()
with g.as_default():
    #Input Data placeholder
    tf_dataset = tf.placeholder(dtype=tf.float32, shape=[None, feature_size])
    tf_labels = tf.placeholder(dtype=tf.float32, shape=[None, num_labels])    
    #Hidden layer
    tf_hidden_weights = tf.Variable(tf.truncated_normal(dtype=tf.float32, shape=[feature_size, hidden_nodes]))
    tf_hidden_bias = tf.Variable(tf.truncated_normal(dtype=tf.float32, shape=[hidden_nodes]))
    tf_hidden_in = tf.matmul(tf_dataset, tf_hidden_weights) + tf_hidden_bias
    tf_hidden_out =tf.nn.relu(tf_hidden_in)
    #Outpu layer
    tf_out_weights = tf.Variable(tf.truncated_normal(dtype=tf.float32, shape=[hidden_nodes, num_labels]))
    tf_out_bias = tf.Variable(tf.truncated_normal(dtype=tf.float32, shape=[num_labels]))
    tf_out_in = tf.matmul(tf_hidden_out, tf_out_weights) + tf_out_bias 
    #Loss, Optimizer and Predictin
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=tf_out_in))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)    
    prediction = tf.nn.softmax(tf_out_in)      
    
#Train and Test
t1 = time.clock()
num_steps = 1000
train_data_length = len(train_x)
with tf.Session(graph = g) as session:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (len(train_y) - batch_size)    
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size)]
        
        train_feed = {tf_dataset : batch_data, tf_labels : batch_labels}
        #Train
        _, train_loss, train_predictions = session.run([optimizer, loss, prediction], feed_dict = train_feed)
        if step%50 == 0:
            print("Minibatch loss at step %d: %f" % (step, train_loss))            
            print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
            #Valid
            valid_feed = {tf_dataset : valid_x, tf_labels : valid_y}
            valid_predictions = session.run(prediction, feed_dict = valid_feed)            
            print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_y))
    #Test
    test_feed = {tf_dataset : test_x, tf_labels : test_y}
    test_predictions = session.run(prediction, feed_dict = test_feed)
    print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_y))
t2 = time.clock()
print('time: ', str(t2-t1))

Minibatch loss at step 0: 10.296930
Minibatch accuracy: 47.7%
Validation accuracy: 54.4%
Minibatch loss at step 50: 7.134218
Minibatch accuracy: 50.8%
Validation accuracy: 55.0%
Minibatch loss at step 100: 1.444922
Minibatch accuracy: 60.9%
Validation accuracy: 56.6%
Minibatch loss at step 150: 1.472411
Minibatch accuracy: 61.7%
Validation accuracy: 52.2%
Minibatch loss at step 200: 1.054441
Minibatch accuracy: 60.9%
Validation accuracy: 56.7%
Minibatch loss at step 250: 0.594066
Minibatch accuracy: 75.8%
Validation accuracy: 56.2%
Minibatch loss at step 300: 0.850249
Minibatch accuracy: 68.8%
Validation accuracy: 56.8%
Minibatch loss at step 350: 0.692948
Minibatch accuracy: 67.2%
Validation accuracy: 54.3%
Minibatch loss at step 400: 0.569166
Minibatch accuracy: 77.3%
Validation accuracy: 56.9%
Minibatch loss at step 450: 0.475952
Minibatch accuracy: 79.7%
Validation accuracy: 56.9%
Minibatch loss at step 500: 0.330795
Minibatch accuracy: 82.0%
Validation accuracy: 56.8%
Minibatch lo